In [2]:
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras import models, layers

2023-09-15 18:57:32.717702: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 18:57:32.890688: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 18:57:32.891609: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 18:57:34.054081: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [4]:
MAX_WORDS_IN_SENTENCE = 700
WORD2VEC_DIMENSIONS = 300

In [5]:
embeddings_index = {}
with open('/home/tina/Downloads/glove.6B.300d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [6]:
def text_to_word_vectors(text, vector_space_dimension):    
    sentence_vectors = []
    for word in text.lower().split():
        if word in embeddings_index and len(sentence_vectors) < MAX_WORDS_IN_SENTENCE:
            sentence_vectors.append(embeddings_index[word])    
    if len(sentence_vectors) > 0:
        pad_len = vector_space_dimension-len(sentence_vectors)      
        if pad_len > 0:
            padding = [np.zeros_like(sentence_vectors[0])] * pad_len
            sentence_vectors = sentence_vectors + padding
    return sentence_vectors

def clean_dataframe(df):
    cleaned_df = df.copy()
    pattern = r'[^a-zA-Z\s]'
    cleaned_df['text'] = cleaned_df['text'].astype(str).apply(lambda x: re.sub(pattern, '', x))    
    return cleaned_df

In [7]:
dataset = pd.read_csv('./data/recipe-dataset.csv')
dataset = clean_dataframe(dataset)
texts = dataset['text'].astype(str)
labels = dataset['label']

In [8]:
data = [text_to_word_vectors(text, MAX_WORDS_IN_SENTENCE) for text in texts]
data = np.array(data)
targets = np.array(labels).astype("float32")

In [9]:
test_x = data[:1500]
test_x =  test_x.reshape(len(test_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
test_y = targets[:1500]

train_x = data[1500:]
train_x = train_x.reshape(len(train_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
train_y = targets[1500:]

In [10]:
model = models.Sequential()
model.add(layers.Dense(50, activation="relu", input_shape=(MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS, )))  
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid")) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                10500050  
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10505201 (40.07 MB)
Trainable params: 1050

2023-09-15 18:58:11.388469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-15 18:58:11.389843: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-09-15 18:58:11.586138: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 42000000 exceeds 10% of free system memory.
2023-09-15 18:58:11.610211: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 42000000 exceeds 10% of free sys

In [11]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_data=(test_x, test_y))

2023-09-15 18:58:13.927628: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1535520000 exceeds 10% of free system memory.


Epoch 1/5


2023-09-15 18:58:15.241134: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 42000000 exceeds 10% of free system memory.


58/58 [==============================] - 10s 167ms/step - loss: 0.0733 - accuracy: 0.9825 - val_loss: 0.0070 - val_accuracy: 0.9980
Epoch 2/5
58/58 [==============================] - 7s 120ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.0061 - val_accuracy: 0.9987
Epoch 3/5
58/58 [==============================] - 7s 128ms/step - loss: 8.9773e-04 - accuracy: 0.9995 - val_loss: 0.0264 - val_accuracy: 0.9947
Epoch 4/5
58/58 [==============================] - 8s 131ms/step - loss: 3.4342e-04 - accuracy: 1.0000 - val_loss: 0.0166 - val_accuracy: 0.9973
Epoch 5/5
58/58 [==============================] - 7s 127ms/step - loss: 1.7531e-04 - accuracy: 1.0000 - val_loss: 0.0163 - val_accuracy: 0.9973


In [13]:
model.save('./data/keras-w2v-recipe-300d.mdl')
loaded_model = models.load_model('./data/keras-w2v-recipe-300d.mdl')
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

INFO:tensorflow:Assets written to: ./data/keras-w2v-recipe-300d.mdl/assets


INFO:tensorflow:Assets written to: ./data/keras-w2v-recipe-300d.mdl/assets


Loaded Model Accuracy: 99.73%


In [15]:
text = '''
Yukon Gold potatoes or Klondike Goldust potatoes – or use Russets
Mayonnaise – your favorite brand
Refrigerated pickle relish 
Yellow mustard – or 1 part yellow + 1 part dijon
Apple cider vinegar
Celery seeds
Paprika
Hard-boiled eggs – peeled and chopped
Celery stalks – diced
Sweet onion – diced
Fresh chopped dill
Salt and pepper
'''
v = np.array(text_to_word_vectors(text, MAX_WORDS_IN_SENTENCE))
v = v.reshape(MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS,1)
t = tf.convert_to_tensor([v])

loaded_model.predict(t)

1/1 [==============================] - 0s 18ms/step


array([[1.]], dtype=float32)